# E6 인공지능 작사가 만들기

## ?? 프로그램 목적 : 인공지능으로 멋진 작사 해 보기
            
### (정규표현식 - 데이터 다듬기, Embadding, LSTM으로 학습 시키기, 잘 만들어 졌는지 평가하기)

### Step 1,2. 데이터 준비하기

In [1]:
import glob
import os
import re
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split # 데이터셋 분리 


txt_file_path = os.getenv('HOME')+'/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)
raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        print(txt_file)
        print("데이터 크기:", len(raw))
        print("Examples:\n", raw[:5])
        print("\n")
        raw_corpus.extend(raw)
        

/home/aiffel-dj29/lyricist/data/lyrics/dr-seuss.txt
데이터 크기: 1005
Examples:
 ['The Cat in the Hat', 'By Dr. Seuss', 'The sun did not shine.', 'It was too wet to play.', 'So we sat in the house']


/home/aiffel-dj29/lyricist/data/lyrics/eminem.txt
데이터 크기: 6812
Examples:
 ['Look, I was gonna go easy on you and not to hurt your feelings', "But I'm only going to get this one chance", "Something's wrong, I can feel it (Six minutes, Slim Shady, you're on)", "Just a feeling I've got, like something's about to happen, but I don't know what", "If that means, what I think it means, we're in trouble, big trouble,"]


/home/aiffel-dj29/lyricist/data/lyrics/drake.txt
데이터 크기: 4773
Examples:
 ['[Hook]', "I've been down so long, it look like up to me", 'They look up to me', "I got fake people showin' fake love to me", 'Straight up to my face, straight up to my face']


/home/aiffel-dj29/lyricist/data/lyrics/al-green.txt
데이터 크기: 2130
Examples:
 ["Let's stay together I, I'm I'm so in love with you", 'Wha

## !! 폴더에 들어 있는 각 파일의 형태를 보고 싶어서 5개씩 출력 해 보니 제거해야할 부분이 보인다

## Step 3. 데이터 정제
### Key point : preprocess_sentence() 함수 사용, 지나치게 긴 문장은  제거,  토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외하기

In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)    # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)           # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>' # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    return sentence

# 문장길이가 100이상으로 너무 길면 제외
corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if len(sentence) > 100: continue   # 길이가 100인 문장은 건너뜁니다. 토큰 15개와 비슷한 길이
        
    corpus.append(preprocess_sentence(sentence))

# 10개 정도 출력 해보자
print("제외 전 문장수 : ", len(raw_corpus))
print("제외 후 문장수 : ", len(corpus))

corpus[:10]


제외 전 문장수 :  187088
제외 후 문장수 :  174176


['<start> the cat in the hat <end>',
 '<start> by dr . seuss <end>',
 '<start> the sun did not shine . <end>',
 '<start> it was too wet to play . <end>',
 '<start> so we sat in the house <end>',
 '<start> all that cold cold wet day . <end>',
 '<start> i sat there with sally . <end>',
 '<start> we sat there we two . <end>',
 '<start> and i said how i wish <end>',
 '<start> we had something to do ! <end>']

### 자 이제 토큰화

In [3]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus) 

    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2    6  900 ...    0    0    0]
 [   2  120 2638 ...    0    0    0]
 [   2    6  300 ...    0    0    0]
 ...
 [   2  135    4 ...    0    0    0]
 [   2  122  169 ...    0    0    0]
 [   2  135    4 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fbb5b652f10>


## Step 4. 평가 데이터셋 분리

### 데이터셋 분리를 위한 텐서 분리

In [4]:
src_input = tensor[:, :-1]   # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성 마지막 토큰은 <end>가 아니라 <pad>일 가능성
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2    6  900   14    6 1335    3    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
[   6  900   14    6 1335    3    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]


### 데이터셋 분리

In [5]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                  tgt_input,
                                                  test_size = 0.2,  # 총 데이터의 20%
                                                  shuffle = True
                                                 )

print("ENC Train: ", enc_train.shape)
print("DEC Train: ", dec_train.shape)
print("ENC Test: ", enc_val.shape)
print("DEC Test: ", dec_val.shape)

ENC Train:  (139340, 34)
DEC Train:  (139340, 34)
ENC Test:  (34836, 34)
DEC Test:  (34836, 34)


### 하이퍼 파라메타 정의

In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
#STEPS_PER_EPOCH = len(enc_train) // BATCH_SIZE
#VALIDATION_STEPS = len(enc_val) // BATCH_SIZE
print("buffer size : ", BUFFER_SIZE)

buffer size :  174176


In [11]:
# tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1    

#dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
#dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
#dataset

train_data = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
train_data = train_data.batch(BATCH_SIZE, drop_remainder=True)
test_data = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
test_data = test_data.batch(BATCH_SIZE, drop_remainder=True)
train_data, test_data

(<BatchDataset shapes: ((256, 34), (256, 34)), types: (tf.int32, tf.int32)>,
 <BatchDataset shapes: ((256, 34), (256, 34)), types: (tf.int32, tf.int32)>)

## Step 5. 인공지능 만들기

### 모델 구축

In [14]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        # 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔줍니다. 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

vocab_size = VOCAB_SIZE
embedding_size = 256  
hidden_size = 1024
model = TextGenerator(vocab_size, embedding_size , hidden_size)

In [18]:
model.summary()

Model: "text_generator_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_4 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_5 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_2 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [17]:
for src_sample, tgt_sample in train_data.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 34, 12001), dtype=float32, numpy=
array([[[-7.7990626e-05,  3.7708956e-05, -8.3427964e-05, ...,
          1.3166625e-04, -1.9439554e-04,  7.7856967e-05],
        [-1.5485342e-04,  1.6902464e-04, -2.6384200e-04, ...,
          1.6632148e-04, -2.8648489e-04, -7.9584912e-05],
        [-2.6284379e-04,  2.1243877e-04, -3.4325288e-04, ...,
          3.0686596e-04, -1.5607038e-04, -1.9492916e-04],
        ...,
        [-1.2399342e-03, -1.4065544e-03,  3.3599245e-03, ...,
         -2.0883812e-03, -2.3864559e-03, -4.4734902e-03],
        [-1.2829560e-03, -1.4388468e-03,  3.3879909e-03, ...,
         -2.1410210e-03, -2.3958413e-03, -4.4807415e-03],
        [-1.3188202e-03, -1.4653449e-03,  3.4092118e-03, ...,
         -2.1863144e-03, -2.4027601e-03, -4.4859629e-03]],

       [[-7.7990626e-05,  3.7708956e-05, -8.3427964e-05, ...,
          1.3166625e-04, -1.9439554e-04,  7.7856967e-05],
        [-4.8431815e-05, -5.1512485e-05,  1.0089142e-04, ...,
          2.4484363e-04, 

### 모델 학습 시작

In [19]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(train_data, validation_data = test_data, epochs=30)

Epoch 1/30
544/544 [==============================] - 160s 293ms/step - loss: 1.6688 - val_loss: 1.4410
Epoch 2/30
544/544 [==============================] - 160s 293ms/step - loss: 1.3900 - val_loss: 1.3602
Epoch 3/30
544/544 [==============================] - 160s 294ms/step - loss: 1.3170 - val_loss: 1.3075
Epoch 4/30
544/544 [==============================] - 161s 296ms/step - loss: 1.2609 - val_loss: 1.2698
Epoch 5/30
544/544 [==============================] - 162s 297ms/step - loss: 1.2137 - val_loss: 1.2399
Epoch 6/30
544/544 [==============================] - 162s 298ms/step - loss: 1.1712 - val_loss: 1.2134
Epoch 7/30
544/544 [==============================] - 163s 299ms/step - loss: 1.1319 - val_loss: 1.1918
Epoch 8/30
544/544 [==============================] - 163s 300ms/step - loss: 1.0946 - val_loss: 1.1739
Epoch 9/30
544/544 [==============================] - 164s 301ms/step - loss: 1.0592 - val_loss: 1.1578
Epoch 10/30
544/544 [==============================] - 165s 303m

### Step.final : 모델 테스트 하기

In [20]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [25]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love it when you call me big poppa <end> '

### 루브릭 평가

1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?
	텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?
    >> 전체적인 문장으로 결과가 나왔음.
    
2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?
	특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?
    >> 데이터 다듬기 완료 !!

3. 텍스트 생성모델이 안정적으로 학습되었는가?
	텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?
    >> val_loss 가 1.22로 2.2 이하 이다 음하하하

### 느낀점과 회고

 1. 데이터 분리하는 방법이 split()으로 진행하여 dataset으로 연결하려는데 문제가 많았고 오래 걸렸음
 2. 데이터 다듬는 방법도 아주 간격하게 잘 설명이 되어 있어서 향후 자주 사용할 것 같음.
 3. 학습 모델으로 rnn을 처음 사용해 보았고 텍스트의 임베딩이 많아서 그런지 파라메터 값이 많이 나왔음. layer가 적지만.... 